In [17]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [18]:
data = pd.read_csv("UpdatedResumeDataSet.csv")

In [19]:
data.dropna(inplace=True)
X = data['Resume']
y = data['Category']

In [20]:
tfidf = TfidfVectorizer(max_features=500)
X_tfidf = tfidf.fit_transform(X)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [22]:
def clean_text(text):
    # Remove non-ASCII characters (such as unwanted encoding issues like Ã¢Â€Â¢)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

# Apply the cleaning function to all string entries in the DataFrame
data_cleaned = data.applymap(lambda x: clean_text(str(x)) if isinstance(x, str) else x)

<ipython-input-22-921e9c17fb2c>:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_cleaned = data.applymap(lambda x: clean_text(str(x)) if isinstance(x, str) else x)


In [23]:
def extract_skills(resume_text):
    # Combine all text in the 'Resume' column
    combined_text = ' '.join(data['Resume'].dropna()).lower()

    # Clean the combined text (remove special characters)
    combined_text = re.sub(r'[^\w\s]', '', combined_text)

    # Split the combined text into unique words (potential skill keywords)
    dynamic_skills = set(combined_text.split())

    # Extract words from the resume text
    words_in_resume = re.findall(r'\b\w+\b', resume_text.lower())

    # Match words from the resume with the dynamic skills list
    extracted_skills = [word for word in words_in_resume if word in dynamic_skills]

    return ', '.join(set(extracted_skills))

In [24]:
def analyze_resume(resume_text):
    # Step 1: Extract skills from the resume
    skills = extract_skills(resume_text)

    # Step 2: Match skills to job categories
    if skills:
        skills_tfidf = tfidf.transform([skills])
        predicted_category = model.predict(skills_tfidf)[0]

        # Step 3: Generate a report
        report = {
            "Extracted Skills": skills,
            "Predicted Job Category": predicted_category
        }
    else:
        report = {
            "Extracted Skills": "No matching skills found",
            "Predicted Job Category": "N/A"
        }
    return report

In [25]:
resume_text = """
CAD, Automobile
"""
resume_report = analyze_resume(resume_text)

# Print Resume Analysis Report
print("\nResume Analysis Report:")
for key, value in resume_report.items():
    print(f"{key}: {value}")



Resume Analysis Report:
Extracted Skills: automobile, cad
Predicted Job Category: Advocate
